In [9]:
# imports
# imports
import pandas as pd
import requests
import os # use this to access your environment variables
FOURSQUARE_KEY = os.getenv('FOURSQUARE_KEY')
YELP_KEY = os.getenv('YELP_KEY')
%store -r df_bike_station_data


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
def get_venues_fs(latitude, longitude, radius, api_key, categories):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        'll': f"{latitude},{longitude}",
        'radius': radius,
        'v': '20220101',  
    }

    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }

    if categories:
        params['categoryId'] = categories

    response = requests.request("GET", url, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.json()


radius=1000
categories=None

# Initialize a dictionary to store results
result_dict = {}

# Function to process each row and save the result
def process_row(row):
    result = get_venues_fs(row['Latitude'], row['Longitude'], radius, FOURSQUARE_KEY, categories)
    result_dict[row.name] = result

# Apply the function to the entire DataFrame
data.apply(process_row, axis=1)

# Display the result dictionary
result_dict
first_poi = result_dict[0]['results'][0]

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
parsed_fourSquare_data = {}

for key, value in result_dict.items():
  venues_data = value.get('results', [])
  parsed_venues = []

  for venue in venues_data:
    venue_name = venue.get('name', 'N/A')
    category_info = venue.get('categories', [])
        
    if category_info:
        category_name = category_info[0].get('name', 'N/A')
    else:
      category_name = 'N/A'
        
    distance = venue.get('distance', 'N/A')

    # Create a dictionary with the extracted information
    parsed_venue = {
      'name': venue_name,
      'category': category_name,
      'distance': distance
    }

    parsed_venues.append(parsed_venue)

  # Store the parsed information in the new dictionary
  parsed_fourSquare_data[key] = parsed_venues

# Display the parsed data
# for key, value in parsed_fourSquare_data.items():
#   print(f"Key: {key}")
#   print(value)
  

#Printing the first 5 surrounding venues from choosen bikeshop
print(f"Key: {1}")
results = parsed_fourSquare_data[1][:5]
for result in results:
   print(result)

Put your parsed results into a DataFrame

In [ ]:
# Create an empty list to store dictionaries
data_list = []

# Iterate through each key in the dictionary
for key in parsed_fourSquare_data:
  # Extract the list of results from the dictionary
  results_list = parsed_fourSquare_data[key]
    
  # Iterate through each result in the list
  for result in results_list:
    # Extract relevant information
    name = result.get('name', 'N/A')
    category = result.get('category', 'N/A')
    distance = result.get('distance', 'N/A')

    #Append the data to the list as a dictionary, including the key
    data_list.append({'Key': key, 'Name': name, 'Category': category, 'Distance': distance})

# Create a DataFrame from parsed_fourSquare_data
df_fourSquare_resultData = pd.DataFrame(data_list)

# Display the DataFrame
%store df_fourSquare_resultData
df_fourSquare_resultData

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# API constants
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
SEARCH_LIMIT = 10


def get_venues_yelp(latitude, longitude, radius, api_key):
    url = f"{API_HOST}{SEARCH_PATH}"
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'categories': '',
        'sort_by': 'best_match',
        'limit': SEARCH_LIMIT
    }

    headers = {
        'Authorization': f'Bearer {api_key}',
        'accept': 'application/json',
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return 'Not Successful'

 
radius = 1000


# Initialize a dictionary to store results
yelp_result_dict = {}

# Function to process each bike shop location and save the result
def yelp_process_row(row):
    yelp_result = get_venues_yelp(row['Latitude'], row['Longitude'], radius, YELP_KEY)
    yelp_result_dict[row.name] = yelp_result

# Apply the function to the entire bikeshop data
data.apply(yelp_process_row, axis=1)

# Display the first poi from yelp dictionary
first_poi = yelp_result_dict[0]['businesses'][0]
print(first_poi)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
parsed_yelp_data = {}

for key, value in yelp_result_dict.items():
  yelp_venues_data = value.get('businesses', [])
  yelp_parsed_venues = []

  for venue in yelp_venues_data:
    yelp_venue_name = venue.get('name', 'N/A')
    category_info = venue.get('categories', [])
    
    if category_info:
        category_title = category_info[0].get('title', 'N/A')
    else:
        category_title = 'N/A'
    
    distance = venue.get('distance', 'N/A')
    rating = venue.get('rating', 'N/A')

    # Create a dictionary with the extracted information
    yelp_parsed_venue = {
        'name': yelp_venue_name,
        'category': category_title,
        'distance': distance,
        'rating': rating
    }

    yelp_parsed_venues.append(yelp_parsed_venue)

  # Store the parsed information in the new dictionary
  parsed_yelp_data[key] = yelp_parsed_venues

# Display the parsed data
# for key, value in parsed_yelp_data.items():
#     print(f"Key: {key}")
#     print(value)

#Printing the first 5 surrounding venues from choosen bikeshop
print(f"Key: {1}")
y_results = parsed_yelp_data[1][:5]
for result in y_results:
  print(result)

Put your parsed results into a DataFrame

# Create an empty list to store dictionaries
yelp_data_list = []

# Iterate through each key in the dictionary
for key in parsed_yelp_data:
  # Extract the list of results from the dictionary
  yelp_results_list = parsed_yelp_data[key]
    
  # Iterate through each result in the list
  for result in yelp_results_list:
    # Extract relevant information
    name = result.get('name', 'N/A')
    category = result.get('category', 'N/A')
    distance = result.get('distance', 'N/A')
    rating = result.get('rating', 'N/A')

    #Append the data to the list as a dictionary, including the key
    yelp_data_list.append({'Key': key, 'Name': name, 'Category': category, 'Distance': distance, 'Rating': rating})

# Create a DataFrame from parsed_yelp_data
df_yelp_resultData = pd.DataFrame(yelp_data_list)


%store df_yelp_resultData
df_yelp_resultData

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

YELP API provided more complete data. YELP API provided ratings and more information on the location that foursquare failed to provide such as rating, price review count, aliases and more.


Get the top 10 restaurants according to their rating

In [ ]:
top_10_restaurants = df_yelp_resultData.sort_values(by='Rating', ascending=False).head(10)
print('Top 10 Restaurants according to rating')
print(top_10_restaurants)